<a href="https://colab.research.google.com/github/sarang-manohar/hogwarts/blob/main/academia/sig788/code/task_4_1_P_Sarang_Manohar(s223504903)_Part2_v1_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install necessary libraries
!pip install fiftyone
!pip install azure-cognitiveservices-vision-customvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 74.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 KB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.9/68.9 KB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.8/37.8 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.2/568.2 KB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 KB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 KB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 KB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 KB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 KB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.5/192.5 KB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [2]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials
import os, time, uuid

In [3]:
import json

with open('creds.txt') as f:
    contents = f.read()
    creds = json.loads(contents)

In [4]:
credentials = ApiKeyCredentials(in_headers={"Training-key": creds['training_key']})
trainer = CustomVisionTrainingClient(creds['ENDPOINT'], credentials)
prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": creds['prediction_key']})
predictor = CustomVisionPredictionClient(creds['ENDPOINT'], prediction_credentials)

In [ ]:
publish_iteration_name = "detect-Model"

# Find the object detection domain
obj_detection_domain = next(domain for domain in trainer.get_domains() if domain.type == "ObjectDetection" and domain.name == "General")

# Create a new project
print ("Creating project...")
# Use uuid to avoid project name collisions.
project = trainer.create_project(str(uuid.uuid4()), domain_id=obj_detection_domain.id)
project_id = project.id

In [ ]:
cls = "bottle"
tag = trainer.create_tag(project_id, cls)

In [ ]:
# Import fiftyone
import fiftyone.zoo as fo

# Import data dataset 
def load_dataset(max_samples, seed):
    dataset_train = fo.load_zoo_dataset(
        "coco-2017",
        split="train",
        label_types=["detections"],
        classes=[cls],
        max_samples=max_samples,
        seed=seed,
        dataset_name="oi-bottle-train" + str(seed),
    )
    return dataset_train

In [ ]:
dataset_train = load_dataset(max_samples=500, seed=2)

INFO:fiftyone.zoo.datasets:Downloading split 'train' to '/root/fiftyone/coco-2017/train' if necessary


Found annotations at '/root/fiftyone/coco-2017/raw/instances_train2017.json'


INFO:fiftyone.utils.coco:Found annotations at '/root/fiftyone/coco-2017/raw/instances_train2017.json'


185 images found; downloading the remaining 315


INFO:fiftyone.utils.coco:185 images found; downloading the remaining 315


...

INFO:fiftyone.zoo.datasets:Loading 'coco-2017' split 'train'


 100% |█████████████████| 500/500 [4.8s elapsed, 0s remaining, 83.4 samples/s]       


INFO:eta.core.utils: 100% |█████████████████| 500/500 [4.8s elapsed, 0s remaining, 83.4 samples/s]       


Dataset 'oi-bottle-train2' created


INFO:fiftyone.zoo.datasets:Dataset 'oi-bottle-train2' created


In [ ]:
import fiftyone as fo
from fiftyone.types.dataset_types import COCODetectionDataset

# Get the path to the annotations file
annFile = '/root/fiftyone/coco-2017/train/labels.json'

from pycocotools.coco import COCO

# Load the annotations file
coco = COCO(annFile)

# Get all the annotations in the dataset
annIds = coco.getAnnIds()
anns = coco.loadAnns(annIds)

# Convert each annotation to the left, top, width, height format
for ann in anns:
    x, y, w, h = ann['bbox']
    left = x
    top = y
    width = w
    height = h
    ann['bbox'] = [left, top, width, height]

import json
# Save the modified annotations file
with open('/root/fiftyone/coco-2017/train/updated_annotation.json', 'w') as f:
    json.dump(coco.dataset, f)

loading annotations into memory...
Done (t=0.18s)
creating index...
index created!


In [ ]:
with open('/root/fiftyone/coco-2017/train/updated_annotation.json') as f:
    contents = f.read()
    annotations = json.loads(contents)
    
import pandas as pd

tmp = pd.DataFrame(annotations['images'])
tmp = tmp[['file_name','height','width','id']].copy()
tmp2 = pd.DataFrame(annotations['annotations'])
tmp2 = tmp2[['image_id','bbox','area','category_id']].copy()
tmp2.rename(columns={'image_id':'id'}, inplace=True)

#merge dataframes
merged_data = pd.merge(tmp,tmp2, how='inner')
tmp3 = pd.DataFrame(annotations['categories'])
tmp3 = tmp3[tmp3['name']=='bottle'].copy()
tmp3.rename(columns={'id':'category_id'}, inplace=True)
merged_data = pd.merge(merged_data,tmp3, how='inner')
merged_data[['Left','Top','Width','Height']] = pd.DataFrame(merged_data.bbox.tolist(), index= merged_data.index)
merged_data['Left'] = merged_data['Left'] / merged_data['width'] 
merged_data['Top'] = merged_data['Top'] / merged_data['height'] 
merged_data['Width'] = merged_data['Width'] / merged_data['width'] 
merged_data['Height'] = merged_data['Height'] / merged_data['height'] 
data_to_upload = merged_data[['file_name','Left','Top','Width','Height']] 
dict = data_to_upload.set_index('file_name').T.to_dict('list')

<ipython-input-42-a6fb87fe2d4c>:25: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  dict = data_to_upload.set_index('file_name').T.to_dict('list')


In [ ]:
data_dir = os.path.join('/root/fiftyone/coco-2017/train/data')

data = pd.DataFrame(dict)

In [ ]:
batch_size = 20
for i in range(0, data.shape[1], batch_size):
    if i != 0:
            print(i," files uploaded so far")
    # Get the next batch of files
    batch_df = data.iloc[:,i:i+batch_size]
    tagged_images_with_regions = []


    for file_name in batch_df.to_dict().keys():
        x,y,w,h = batch_df[file_name]
        regions = [ Region(tag_id=tag.id, left=x,top=y,width=w,height=h) ]

        with open(os.path.join(data_dir, file_name ), mode="rb") as image_contents:
            tagged_images_with_regions.append(ImageFileCreateEntry(name=file_name, contents=image_contents.read(), regions=regions))

        upload_result = trainer.create_images_from_files(project_id, ImageFileCreateBatch(images=tagged_images_with_regions))

20  files uploaded so far
40  files uploaded so far
60  files uploaded so far
80  files uploaded so far
100  files uploaded so far
120  files uploaded so far
140  files uploaded so far
160  files uploaded so far
180  files uploaded so far
200  files uploaded so far
220  files uploaded so far
240  files uploaded so far
260  files uploaded so far
280  files uploaded so far
300  files uploaded so far
320  files uploaded so far
340  files uploaded so far
360  files uploaded so far
380  files uploaded so far
400  files uploaded so far
420  files uploaded so far
440  files uploaded so far
460  files uploaded so far
480  files uploaded so far


# Train the project

In [ ]:
iteration = trainer.train_project(project_id)
while (iteration.status != "Completed"):
    iteration = trainer.get_iteration(project_id, iteration.id)
    print ("Training status: " + iteration.status)
    time.sleep(1)

Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training sta

In [ ]:
# The iteration is now trained. Publish it to the project endpoint
trainer.publish_iteration(project_id, 'e038becd-50b5-4719-a39a-1805a19ad4dc', 'Iteration 1', '/subscriptions/7e549871-41a3-4163-af83-5546a9679c54/resourceGroups/sig788-task4/providers/Microsoft.CognitiveServices/accounts/leafclass')

Done!


In [28]:
#Get project list
projects = trainer.get_projects()

project_list = []
for project in projects:
    project_list.append(project.id)
project_id=project_list[1]
project_id

'e9c32e6a-a241-401c-bc7f-e859fb047664'

In [11]:
prediction_url = 'https://leafclass-prediction.cognitiveservices.azure.com/customvision/v3.0/Prediction/e9c32e6a-a241-401c-bc7f-e859fb047664/detect/iterations/Iteration2/image'

In [27]:
from PIL import Image, ImageDraw, ImageFont
# Set the threshold for the probability of the predictions
THRESHOLD = 0.25

# Set the font for drawing the tag labels
FONT = ImageFont.load_default()

# Set the tag colors dictionary
TAG_COLORS = {
    "person": (253, 1, 0),
    "bicycle": (247, 105, 21),
    "motorcycle": (238, 222, 4),
    "bus": (47, 162, 54),
    "car": (51, 62, 212)
}

# Define the draw_boxes function
def draw_boxes(image, predictions, tag_colors):
    draw = ImageDraw.Draw(image)
    for prediction in predictions:
        tag = prediction["tagName"]
        probability = prediction["probability"]
        if probability >= THRESHOLD:
            box = prediction["boundingBox"]
            left = int(box["left"] * width)
            top = int(box["top"] * height)
            right = int((box["left"] + box["width"]) * width)
            bottom = int((box["top"] + box["height"]) * height)
            draw.rectangle([(left, top), (right, bottom)], outline=tag_colors.get(tag, "red"), width=2)
            draw.text((left, top-20), f"{tag} {probability:.2f}", font=FONT, fill=tag_colors.get(tag, "red"))
    return image

In [25]:
import cv2, requests, json
import numpy as np
test_path = '/content/testImages'
test_files = os.listdir(test_path)

ENDPOINT = prediction_url
KEY = creds['prediction_key']

for images in test_files:
    image_path = f'/content/testImages/{images}'
    # Read the image file with OpenCV
    img = cv2.imread(image_path)

    # Convert the image to RGB format
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  

    # Compress the image as a JPEG image
    _, encoded_frame = cv2.imencode(".jpg", img, [int(cv2.IMWRITE_JPEG_QUALITY), 80])

    # Convert the image to a byte array
    img_bytes = cv2.imencode('.jpg', img)[1].tobytes()

    headers = {
        "Prediction-Key": KEY,
        "Content-Type": 'application/json',
    }
    body = {
        "Data": img.to_list(),
    }
    
    response = requests.post(ENDPOINT, headers=headers, json=body, data=encoded_frame.tobytes())
    response.raise_for_status()
    predictions = response.json()["predictions"]

    # Draw the bounding boxes and labels on the image
    img_with_boxes = draw_boxes(Image.fromarray(img), predictions, TAG_COLORS)
    img_with_boxes = cv2.cvtColor(np.asarray(img_with_boxes), cv2.COLOR_RGB2BGR)  

    #Display the annotated frame
    cv2.imshow("Frame", np.asarray(img_with_boxes))


uint8
uint8
uint8
uint8
uint8


In [ ]:
# Now there is a trained endpoint that can be used to make a prediction

# Open the sample image and get back the prediction results.


# Display the results.    
for prediction in results.predictions:
    print("\t" + prediction.tag_name + ": {0:.2f}% bbox.left = {1:.2f}, bbox.top = {2:.2f}, bbox.width = {3:.2f}, bbox.height = {4:.2f}".format(prediction.probability * 100, prediction.bounding_box.left, prediction.bounding_box.top, prediction.bounding_box.width, prediction.bounding_box.height))